In [10]:
import requests
from bs4 import BeautifulSoup
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import instaloader
from datetime import datetime
import pandas as pd

In [2]:
url1 = "https://www.bellanaijastyle.com/imad-eduso-lagos-fw-2024/"

response1 = requests.get(url1)
soup1 = BeautifulSoup(response1.content, 'html.parser')

images1 = []
for img1 in soup1.find_all('img1'):
    src = img1.get('src')
    if src and 'imad' in src:
        images1.append(src)
print(f"Found {len(images1)} images from Imad Eduso collection.")

Found 0 images from Imad Eduso collection.


In [3]:
os.makedirs('imad_eduso_images', exist_ok=True)

for i, img_url in enumerate(images1):
    img_data = requests.get(img_url).content
    with open(os.path.join('imad_eduso_images', f'imad_eduso_{i+1}.jpg'), 'wb') as f:
        f.write(img_data)

In [4]:
desc = []
for p in soup1.find_all('p', class_='description'):
    text = p.get_text(strip=True)
    if 'Imad Eduso' in text:
        desc.append(text)
    
print(desc[:10])

[]


In [8]:
driver = webdriver.Chrome()
driver.get(url1)

time.sleep(15)
elements = driver.find_elements(By.TAG_NAME, 'img')
print(f"Found {len(elements)} image elements.")

for index, img in enumerate(elements):
    src = img.get_attribute('src')
    if src and src.startswith("http"):
        try:
            img_data = requests.get(src).content
            with open((f'imad_eduso_images{index+1}.jpg'), 'wb') as f:
                f.write(img_data)
        except Exception as e:
            continue

driver.quit()
print("Image download completed.")

Found 46 image elements.
Image download completed.


In [ ]:
IG =instaloader.Instaloader()

IG.load_session_from_file("dijalaanita")

profile = instaloader.Profile.from_username(IG.context, "imadeduso_")

start_date = datetime(2024, 10, 1)
end_date = datetime(2024, 10, 31, 23, 59, 59)

posts = []

for post in profile.get_posts():
    post_date = post.date_utc
    if start_date <= post_date <= end_date:
        print(f"getting posts from {post_date}...")
        captions = post.caption or ""
        imad_insta_url = post.url

        posts.append({
            "date": post_date.strftime("%Y-%m-%d"),
            "caption": captions,
            "url": imad_insta_url
        })
        print("Collected posts")

    time.sleep(2)
print(f"Total posts collected: {len(posts)}")

imad_df = pd.DataFrame(posts)
output_file = f"{'imadeduso_'}_posts_october_2024.csv"
imad_df.to_csv(output_file, index=False, encoding='utf-8')
print(f"Data saved to {output_file}")